In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import seaborn as sns
from gerrychain import Graph, GeographicPartition, Partition, Election, accept
from gerrychain.updaters import Tally, cut_edges
import glob
import functools
import operator

In [2]:
def foldl(func, acc, xs):
  return functools.reduce(func, xs, acc)

foldr = lambda func, acc, xs: functools.reduce(lambda x, y: func(y, x), xs[::-1], acc)

In [3]:
def get_state_runs(seats, iters=100000, pop_bal=2.0, min_col="BVAP",
                   ls=[2,5,10,20,40,80], ps=[0.25, 0.125, 0.0625]):
    results = {}

    for l in ls:
        sb_runs = glob.glob("data/states/{}_dists{}_{}opt_{}%_100000_sbl{}_score0_*.npy".format("GA", seats, # TODO: Is it okay to pass GA here?
                                                                                                min_col, pop_bal, l))
        results[str(l)] = np.zeros((len(sb_runs), iters))
        for i, run in enumerate(sb_runs):
            results[str(l)][i] = np.load(run).flatten()
    for p in ps:
        tilt_runs = glob.glob("data/states/{}_dists{}_{}opt_{}%_100000_p{}_*.npy".format("GA", seats, 
                                                                                         min_col, pop_bal, p))
        results[str(p)] = np.zeros((len(tilt_runs), iters))
        for i, run in enumerate(tilt_runs):
            results[str(p)][i] = np.load(run).flatten()
    return results

In [4]:
la_runs_bvap = get_state_runs("GA", 105, ls=[2,5,10,25,50,100,200], pop_bal=4.5, min_col="BVAP")

In [5]:
# TODO: What is glob? what does it do? What's the path below?
# TODO: Do we need unbiased run for short burst?

ubs = glob.glob("data/unbiased_GA/GA_dists105_4.5%_100000_unbiased_*.p")
ub_runs = {}
for i, run in enumerate(ubs):
     with open(run, "rb") as f:
        ub_runs[i] = pickle.load(f)

In [6]:
runs = list(ub_runs.values())
bvap = foldl(lambda x,y: np.concatenate((x,y["BVAP"]), axis=0), runs[0]["BVAP"], runs[1:])

IndexError: list index out of range